In [ ]:
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn import svm

In [ ]:
data = load_iris()
data.keys()

In [ ]:
data.get('target_names')

In [ ]:
setosa = data.get('data')[:25][:, 1:3]
# y = data.get('target')[:25]
# x = setosa

versicolor = data.get('data')[50:75][:, 1:3]
y = data.get('target')[50:75]
x = versicolor

virginica = data.get('data')[100:125][:, 1:3]
y = np.append(y, data.get('target')[100:125])
x = np.append(x, virginica, axis=0)

In [ ]:
def plot_iris():
    plt.title('Plot three classes on features')
    # plt.scatter(setosa[:, 0], setosa[:, 1], label='setosa')
    plt.scatter(versicolor[:, 0], versicolor[:, 1], label='versicolor')
    plt.scatter(virginica[:, 0], virginica[:, 1], label='virginica')
    plt.xlabel(data.get('feature_names')[0])
    plt.ylabel(data.get('feature_names')[1])
    plt.legend()

In [ ]:
model = svm.SVC(kernel='linear')

In [ ]:
model.fit(x,y)

In [ ]:
model.coef_

In [ ]:
w = -model.coef_[0][0]/model.coef_[0][1]
b = model.intercept_[0]
line_x = np.arange(2,4,0.01)
line_y = line_x * w - b/model.coef_[0][1]

In [ ]:
plot_iris()
plt.scatter(model.support_vectors_[:, 0], model.support_vectors_[:, 1], color='red')
plt.plot(line_x, line_y, linewidth=1)
plt.show()

## Build SVMs using Numpy

### Split datasets

In [ ]:
ratio = 0.8

assert(ratio*len(versicolor), len(virginica))
total_len = len(versicolor)
N = int(ratio*total_len)
train_versicolor, test_versicolor = versicolor[:N], versicolor[N:]
train_virginica, test_virginica = virginica[:N], virginica[N:]


train_x = np.append(train_versicolor, train_virginica, axis=0)
train_y = np.append([[-1 for i in range(N)]], [[1 for i in range(N)]],axis=None)
test_x = np.append(test_versicolor, test_virginica, axis=0)
test_y = np.append([[-1 for i in range(total_len-N)]], [[1 for i in range(total_len-N)]],axis=None)
train_y,test_y


In [ ]:
def hinge_loss(t, type:str = 'soft'):
    if type == 'soft':
        if t >= 0:
            return 0
        else:
            return 1-t
    elif type == 'hard':
        if t >= 1:
            return 0
        else:
            return np.inf
    else:
        raise Exception('Type of SVM not provided. Select `soft` or `hard`')

class SVM():
    def __init__(self, learning_rate: int = 0.00001) -> None:
        self.w = np.array([])
        self.learning_rate = learning_rate        

    def train(self, X: list, Y: list, type: str = 'soft', epochs: int = 100) -> None:
        '''
            Train the SVM using Hinge loss and gradient descent.
            Parameters:
                X (list): training data
                Y (list): training labels
                epochs (int): number of times the whole optimization will be run
        '''
        assert len(X) == len(Y), 'Size of data does not match size of labels'
        assert self.learning_rate > 0, 'Learning rate must be positive'

        # Initialise the weights array to zero
        self.w = np.random.random((1, 2))
        # self.w = np.array([[0.5, 0.5] for i in range(len(X))])

        for epoch in range(epochs):
            Y_hat = np.sum(self.w * X, axis=1)
            t = Y * Y_hat
            # print(t)
            for _ in t:
                # Update weights
                # print(hinge_loss(_))
                print(self.w, hinge_loss(_), self.w/epochs)
                self.w += self.learning_rate * (hinge_loss(_) - 2 * self.w/epochs)



In [ ]:
# Test
t = -1 * 15
t = 1 * 2
hinge_loss(t)

In [ ]:
svm = SVM()
svm.train(X=train_x, Y=train_y)
svm.w

In [ ]:
np.sqrt(svm.w[0, 0]**2 + svm.w[0, 1]**2)

In [ ]:
w = -svm.w[0][1]/svm.w[0][0]
b = train_x[0][0] * svm.w[0][0] - train_x[0][1] * svm.w[0][1]
line_x = np.arange(2,4,0.01)
line_y = line_x * w - b/svm.w[0][1]

In [ ]:
print(svm.w)
print(model.coef_)

In [ ]:
plot_iris()
plt.scatter(svm.w[:, 0], svm.w[:, 1], color='red')
plt.plot(line_x, line_y, linewidth=1)
plt.show()

### Non-linearly separable

In [ ]:
xor_data = np.array([
    [1, 1],
    [2, 2],
    [1, 2],
    [2, 1]
])
xor_labels = np.array([0, 0, 1, 1])

In [ ]:
def plot_xor(xor_data):
    plt.scatter(x=xor_data[:2, 0], y=xor_data[:2, 1])
    plt.scatter(x=xor_data[2:, 0], y=xor_data[2:, 1])
    plt.xlim(-1, 4)
    plt.ylim(-1, 4)
plot_xor(xor_data)

In [ ]:
model = svm.SVC(kernel='rbf')
model.fit(xor_data, y=xor_labels)